In [ ]:
# 0) 필요한 패키지 설치 (한 번만)


In [27]:
import pandas as pd
from pyproj import Transformer
import folium

# 1) CSV 읽기 (한글 인코딩 오류 방지)
df = pd.read_csv("PURPOSE_250M_202403.csv", encoding="cp949")

# 2) 컬럼명에 맞춰 숫자형 변환 & 결측치 제거
df['cell_X_o'] = pd.to_numeric(df['O_CELL_X'], errors='coerce')
df['cell_Y_o'] = pd.to_numeric(df['O_CELL_Y'], errors='coerce')
df['cell_X_d'] = pd.to_numeric(df['D_CELL_X'], errors='coerce')
df['cell_Y_d'] = pd.to_numeric(df['D_CELL_Y'], errors='coerce')
df = df.dropna(subset=['cell_X_o', 'cell_Y_o', 'cell_X_d', 'cell_Y_d']) \
       .reset_index(drop=True)

# 3) Transformer 정의
#   EPSG:5179 → EPSG:3857 → EPSG:4326
t1 = Transformer.from_crs("EPSG:5179", "EPSG:3857", always_xy=True)
t2 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# 4) 출발지 변환
# x_o_3857, y_o_3857 = t1.transform(df['cell_X_o'].values, df['cell_Y_o'].values)
# df['x_o_3857'], df['y_o_3857'] = x_o_3857, y_o_3857  # 3857 컬럼은 데이터에 남기지 않음
lon_o, lat_o = t2.transform(*t1.transform(df['cell_X_o'].values, df['cell_Y_o'].values))
df['lon_o'], df['lat_o'] = lon_o, lat_o

# 5) 도착지 변환
# x_d_3857, y_d_3857 = t1.transform(df['cell_X_d'].values, df['cell_Y_d'].values)
# df['x_d_3857'], df['y_d_3857'] = x_d_3857, y_d_3857  # 3857 컬럼은 데이터에 남기지 않음
lon_d, lat_d = t2.transform(*t1.transform(df['cell_X_d'].values, df['cell_Y_d'].values))
df['lon_d'], df['lat_d'] = lon_d, lat_d

# 6) 결과 CSV로 저장 (utf-8-sig)
df.to_csv("PURPOSE_250M_202403_with_lonlat.csv",
          index=False, encoding="utf-8-sig")

# 7) Folium 대화형 지도 생성 (출발: 파란 점, 도착: 빨간 점)
m = folium.Map(
    location=[df['lat_o'].mean(), df['lon_o'].mean()],
    zoom_start=7,
    tiles="OpenStreetMap"
)

for _, row in df.iterrows():
    # 출발지 (blue)
    folium.CircleMarker(
        location=[row['lat_o'], row['lon_o']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.7,
    ).add_to(m)
    # 도착지 (red)
    folium.CircleMarker(
        location=[row['lat_d'], row['lon_d']],
        radius=2,
        color='red',
        fill=True,
        fill_opacity=0.7,
    ).add_to(m)

m.save("PURPOSE_250M_202403_map.html")
print("✅ 변환 완료:")
print("  • PURPOSE_250M_202403_with_lonlat.csv")
print("  • PURPOSE_250M_202403_map.html")


✅ 변환 완료:
  • PURPOSE_250M_202403_with_lonlat.csv
  • PURPOSE_250M_202403_map.html


In [20]:
print(df[['cell_X','cell_Y']].head())
print(df[['lon','lat']].head())    # folium용 변환 후


   cell_X   cell_Y
0  931125  1946875
1  944875  1950125
2  954125  1942125
3  952375  1951375
4  954375  1916625
          lon        lat
0  137.330121  50.563444
1  137.527406  50.574899
2  137.639654  50.492419
3  137.633983  50.576383
4  137.592244  50.266788


In [22]:
from pyproj import Transformer

# 1) 테스트할 한 점
x_in, y_in = 931125, 1946875

# 2) EPSG:5174 → EPSG:3857
t1 = Transformer.from_crs("EPSG:5174", "EPSG:3857", always_xy=True)
x_3857, y_3857 = t1.transform(x_in, y_in)
print(f"→ EPSG:3857:  x={x_3857:.3f}, y={y_3857:.3f}")

# 3) EPSG:3857 → EPSG:4326
t2 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
lon, lat = t2.transform(x_3857, y_3857)
print(f"→ WGS84:  lon={lon:.6f}, lat={lat:.6f}")


→ EPSG:3857:  x=15287519.136, y=6544432.374
→ WGS84:  lon=137.330121, lat=50.563444
